# This is your first ChatBot

### It is an extension of previous Practicals when you could query a .pdf document, except that now you can have an entire conversation about the document. It uses `LangChain` as a development environmennt. This first Notebook uses a previous version of LangChain, which is good to start with (and still works). In the next Notebook, you will see a more recent version. This first exercise is about porting a previous apporoach to a dialogue context, and getting started with Prompt writing for ChatBots.

In [1]:
%%capture
!pip install python-dotenv
!pip install openai
!pip install langchain==0.1.0
!pip install pdfplumber

import os
import openai
import pdfplumber

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    StringPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate
)

## Import a .pdf file whose contents you want to have a dialogue about. Keep it shorter than 20 pages.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # Now you can work with the uploaded file, e.g., using pdfplumber:
  with pdfplumber.open(fn) as pdf:
    for page in pdf.pages:
      my_doc = page.extract_text()

Saving Thesis Andrea Mejia _MIMM_Final..pdf to Thesis Andrea Mejia _MIMM_Final..pdf
User uploaded file "Thesis Andrea Mejia _MIMM_Final..pdf" with length 2080545 bytes


## You need to author a Prompt that will govern how the Bot behaves. This is the main learning exercise.

In [3]:
prompt = PromptTemplate(
    template="""
Here you describe what the Bot is about in relation to the Document \
e.g. instructor, advisor ...

---
IMPORTANT:
Your reference document for answering questions is in this file:
<REF>
{my_doc}
</REF>
---

Here you describe how the Bot should answer:
- provide detailed responses to your answers
- end each response with a question about the same topic
- answer in a formal style as a critical friend


Current conversation:
{history}

Candidat: {human_input}
Bot: ""
""",
input_variables=["my_doc", "history", "human_input"]
)

prompt_formatted_str: str = prompt.format(
    my_doc= my_doc,
    history="{history}",
    human_input="{human_input}")

prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [8]:
key= *** insert your OPENAI API KEY here ***

### This is the ChatBot core. You are creating a Chain which contains:
*   an LLM (here gpt-4o-mini)
*   your Prompt
*   a memory buffer so that the Bot remembers previous turns in the conversation




In [9]:
llm = ChatOpenAI(openai_api_key=key, model="gpt-4o-mini", temperature=0.7)

memory = ConversationBufferWindowMemory(
    memory_key="history", k=7, return_messages=True)

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

### A minimal User Interface to run the ChatBot. It also logs the dialogue into a file (NOTE: writing to the file is "a" mode).

In [11]:
import ipywidgets as widgets
from IPython.display import display
import textwrap

# Create UI elements
user_input_box = widgets.Text(
    placeholder="Enter your message here...",
    description="User:",
    style={'description_width': 'initial'}
)
chat_output = widgets.Output()
end_chat_button = widgets.Button(description="End Chat", button_style="danger")

# Display UI elements
display(chat_output, user_input_box, end_chat_button)

# Open a file for saving dialogue
Dfile = open("dialogue_log.txt", "a")

def handle_input(text):
    user_input = text.value.strip()

    if not user_input:
        return  # Ignore empty input

    if user_input.lower() in ["exit", "quit", "fin", "bye", "fini"]:
        with chat_output:
            print("Chat ended by user.")
        user_input_box.disabled = True
        return

    # Write user input to file
    print("User: " + user_input, file=Dfile)

    # Process response (replace with actual call to OpenAI API)
    response = chat_llm_chain.predict(human_input=user_input)

    # Wrap text for better display
    wrapped_response = textwrap.fill(response, width=120)

    with chat_output:
        print(f"User: {user_input}")
        print(f"Bot: {wrapped_response}")

    # Write bot response to file
    print("Bot: " + response, file=Dfile)

    # Clear input box for next message
    user_input_box.value = ""

def stop_chat(_):
    with chat_output:
        print("Chat ended by user.")
    user_input_box.disabled = True
    Dfile.close()

# Event bindings
user_input_box.on_submit(handle_input)  # Waits for Enter before triggering
end_chat_button.on_click(stop_chat)


Output()

Text(value='', description='User:', placeholder='Enter your message here...', style=DescriptionStyle(descripti…

Button(button_style='danger', description='End Chat', style=ButtonStyle())

# Step 1 in Bot development is to understand the various elements, and how the ChatBot behaviour depends on its Prompt. Developing an ChatBot involves:
*   Choosing how it accesses specific information
*   Setting up a dialogue context (which is a loop within which completion is applied to each user input)
*   Authoring a Prompt governing dialogue behaviour